# AD9833 Modulators test with FTDI
https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [ ]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [ ]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [ ]:
paths = [['..', '..', '..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [ ]:
%pylab inline 

from signal_generators import modulators 
from signal_generators.ad98xx.ad9833 import *
from signal_generators.ad98xx.ad9850 import * 
from utilities.adapters import peripherals
from utilities.shift_register import ShiftRegister

## Debug mode?

In [ ]:
AD9833.DEBUG_MODE_SHOW_BUS_DATA = False        # whether to show bus data. 
AD9833.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [ ]:
with_hardware_device = True

if with_hardware_device:
    _clk = peripherals.Pin.get_Ftdi_pin(pin_id = 4)
    _data = peripherals.Pin.get_Ftdi_pin(pin_id = 1)

    _ss1 = peripherals.Pin.get_Ftdi_pin(pin_id = 3)
    _ss2 = peripherals.Pin.get_Ftdi_pin(pin_id = 0)

    _spi_1 = ShiftRegister(stb_pin = _ss1, clk_pin = _clk, data_pin = _data, polarity = 1)
    _spi_2 = ShiftRegister(stb_pin = _ss2, clk_pin = _clk, data_pin = _data, polarity = 1)
else:
    _spi_1 = _spi_2 = _ss1 = _ss2 = None  # using None for testing without actual hardware device.


ad1 = AD9833(_spi_1, _ss1)
ad2 = AD9833(_spi_2, _ss2)

In [ ]:
bfsk = modulators.BFSK(ad1)
bpsk = modulators.BPSK(ad1)
dtmf = modulators.DTMF((ad1, ad2))
fm = modulators.FM(ad1)
iq = modulators.IQ((ad1, ad2))
ook = modulators.OOK(ad1)
pm = modulators.PM(ad1)
pwm = modulators.PWM(ad1)
qpsk = modulators.QPSK(ad1)

In [ ]:
ad1.reset()
ad2.reset()

In [ ]:
ad1.enable_output(False)
ad2.enable_output(False)

## Testing data sequence

In [ ]:
samples_size = 10
duration = 0.1

symbols = np.random.randint(2, size = samples_size)
digital_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.uniform(-1, 1, size = samples_size).round(5)
analog_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice(['1', '2', '3', 'A', '4', '5', '6', 'B', '7', '8', '9', 'C', '*', '0', '#', 'D'], 
                           size = samples_size)
dtmf_sequence = [(symbol, duration) for symbol in symbols] 


symbols = np.random.choice([0, 1, 2, 3], size = samples_size)
quadrature_sequence = [(symbol, duration) for symbol in symbols] 

## Modulators test

In [ ]:
def dump(m):
    m._device.dump()

In [ ]:
bfsk.send_sequence(digital_sequence)

In [ ]:
bpsk.send_sequence(digital_sequence)

In [ ]:
ook.send_sequence(digital_sequence)

In [ ]:
fm.send_sequence(analog_sequence)

In [ ]:
pm.send_sequence(analog_sequence)

In [ ]:
qpsk.send_sequence(quadrature_sequence)

In [ ]:
iq.send_sequence(quadrature_sequence)

In [ ]:
dtmf.send_sequence(dtmf_sequence)

In [ ]:
pwm.run()